## Kalman Filter
The aim of this notebook is to understand and experiment with Kalman Filters, as a potential improvement upon the current glicko-2 rating system.

### 1. Background
**Assumptions**

A Kalman filter estimates something that cannot be observed directly. It does so by using:
1. A model of how the target quantity evolves over time
2. Noisy samples of the target quantity

The KF then maintains a belief about the hidden state.

The KF assumes that the hidden state evolves over time, $$x_t = Fx_{t-1} + w_t$$ 

where, F is a transition matrix and $w_t$ is a gaussian process noise.

We also assume that our observations are noisy, $$y_t = Hx_t + v_t$$ 

where, H is an observation matrix, that relates the inner state to our observation. $y_t$ is the observed performance, and $v_t$ is a gaussian observation noise.

**Updates**

The updates occur in two steps.

First we build the prior. We estimate our state to be 
$$\hat{x}_{t|t-1} = F\hat{x}_{t-1}.$$ 

Our uncertainty grows from the previous observation, 
$$P_{t|t-1} = FP_{t-1}F^T + Q.$$

Next, we update the understanding with new information. The Kalman Gain, $K_t$, denotes how much the new observation should be trusted vs the prior, given by $$K_t = P_{t|t-1}H^T(HP_{t|t-1}H^T + R)^{-1}.$$

Based on this Kalman Gain, we update the state and uncertainty:
$$\hat{x_{t}} = \hat{x}_{t|t-1} + K_t(y_t - H\hat{x}_{t|t-1})$$
$$P_t = (I - K_tH)P_{t|t-1}$$

Overall, if the uncertainty is high, we trust new data more. If uncertainty is low, we trust the prior more. Each observation also reduces the uncertainty.

### 2. State Construction

In traditional Elo or Glicko-2 skill measurement techniques, the true skill is a number that moves up or down based only on match results and uncertainty. In reality, the result of a match is a noisy measurement of a player's underlying ability, and is influenced by many other factors, including fatigue, surface preference and more.

The simplest state that represents an observation is to include the true skill of both players:
$$x = [s_a, s_b]^T, \space P = \begin{bmatrix} P_A & 0 \\ 0 & P_B \end{bmatrix}$$

Then we propose that true skill is a gaussian random walk: $$s_t=s_{t-1} + w_t,\space w_t \sim N(0, q)$$

The prior hence is constructed as:
$$x^- = x$$
$$P^- = P + Q = \begin{bmatrix} P_A + q & 0 \\ 0 & P_B + q \end{bmatrix}$$

Here we assume the match result is solely driven by the skill difference between the two players. We model the probability that player A wins the match as
$$\hat{y}^- = h(x^-)$$
where we use the logistic elo function
$$h(x) = (1+\exp(-k\begin{bmatrix} 1 & -1 \end{bmatrix}x))^{-1}$$

However, because $h(x)$ is non linear, we should use the Extended Kalman Filter, and approximate h(x) with the taylor expansion. We thus have,

$$ h(x) \approx  h(x^-) + H(x - x^-)$$
where $H$ is the Jacobian of $h$ evaluated at $x^-$,

$$H = kp^-(1-p^-)\begin{bmatrix} 1 & -1\end{bmatrix},\space p^-=h(x^-).$$

Using the above formulas, we compute the Kalman Gain:
$$S=HP^-H^T + R$$
$$K = P^-H^TS^{-1}$$

Here $R$ represents the noise from the result of the match. Because we approximate the result with 
$$r \approx h(x^-) + v, \space v \sim N(0, R).$$

Because $r \sim Bern(\hat{y}^-)$, a natural choice of $R$ is $R = \hat{y}^-(1-\hat{y}^-)$ to approximate the Bernoulli observation noise by a Gaussian with the same variance.

We then update the posterior
$$x^+ = x^- + Ke, \space e = r - \hat{y}^-$$
$$P^+ = (I - KH)P^-$$
where $r$ is the result. 1 if A wins, 0 if A loses.

